# Project 4 K-Nearest Neighbors On Two Data Sets

## Part 1: Cleveland Data Set

In [ ]:
#Common imports for use in the data visualization
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,  precision_score, recall_score, f1_score


from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
clevelandDf = pd.read_csv('datasets/cleveland.csv')

clevelandDf.replace('?', np.nan, inplace=True)

clevelandDf['num'] = (clevelandDf['num'] > 0).astype(int)

clevelandDf.dropna(inplace=True)

selected_features = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
X = clevelandDf[selected_features]
y = clevelandDf['num']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

best_k = 1  # test others
nn = NearestNeighbors(n_neighbors=best_k, metric='euclidean', algorithm='auto')
nn.fit(X_train)

distances, indices = nn.kneighbors(X_test)
predictions = [np.any(y_train.iloc[indices[i]] > 0) for i in range(len(X_test))]  # Considering any non-zero prediction as heart disease

precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print("K value:", best_k)
print("Selected attributes:", selected_features)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

- As I messed around with different K values, I relaized that as I went up, we got worse. This is because of overfitting. As we made the model more complex than it needs to be, we started getting more and more incorrect. But as our K decreased, our scores went up, leading us to go with a lower K.

In [ ]:
new_data = pd.read_csv('datasets/cleveland-test-sample.csv')

new_data.replace('?', np.nan, inplace=True)
new_data.dropna(inplace=True)
X_new = new_data[selected_features]

X_new_scaled = scaler.transform(X_new)

nn = NearestNeighbors(n_neighbors=best_k, metric='euclidean', algorithm='auto')
nn.fit(X_scaled)
distances, indices = nn.kneighbors(X_new_scaled)
predictions = [np.any(y.iloc[indices[i]] > 0) for i in range(len(X_new_scaled))]  # Consider any non-zero prediction as heart disease

print("Predictions for new dataset:")
print(predictions)

## Part 2: Student Dropout Data Set

## [Good Datasets Here](https://archive.ics.uci.edu/)

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

school_df = pd.read_csv('datasets/data.csv', sep=';')
interested_columns = ["Age at enrollment", "Tuition fees up to date", "Previous qualification (grade)", "Scholarship holder", "Target"]
school_interested = school_df[interested_columns].copy()

dropout_count = school_interested['Target'].value_counts().get('Dropout')
print("Number of dropouts:", dropout_count)
not_dropout_count = (school_interested['Target'] != 'dropout').sum()
print("Number of students who are not dropouts:", not_dropout_count)


# Standardize the non-binary data
school_interested['Age standardized'] = (school_interested['Age at enrollment'] - school_interested['Age at enrollment'].mean()) / (school_interested['Age at enrollment'].std())
school_interested['Tuition standardized'] = (school_interested['Tuition fees up to date'] - school_interested['Tuition fees up to date'].mean()) / (school_interested['Tuition fees up to date'].std())
school_interested['Grades standardized'] = (school_interested['Previous qualification (grade)'] - school_interested['Previous qualification (grade)'].mean()) / (school_interested['Previous qualification (grade)'].std())
school_interested['Scholarship standardized'] = (school_interested['Scholarship holder'] - school_interested['Scholarship holder'].mean()) / (school_interested['Scholarship holder'].std())

school_interested = school_interested.sort_values(by='Scholarship holder', ascending=False)

nn = NearestNeighbors(n_neighbors=9, metric='euclidean', algorithm='auto')

student = school_interested.sample(1)
studentX = student[['Age standardized', 'Grades standardized']].values[0]		# moved this block above
display(studentX)

# need to get rid of the patient itself
X = school_interested.drop(student.index)[['Age standardized', 'Grades standardized']].values

# Build index data structure under the hood for query performance
fit = nn.fit(X)

# Find k nearest neighbors
distances, indices = fit.kneighbors([studentX])
distances, indices

# Get the patients that are near the age
nbrs = school_interested.iloc[indices[0]]
display(nbrs)

# Print how many patients are sick and how many are healthy
in_school = nbrs[(nbrs['Target'] == "Enrolled") | (nbrs['Target'] == "Graduate")].count()
dropout = nbrs[nbrs['Target'] == "Dropout"].count()
#print('in school: {}\ndropout: {}'.format(in_school, dropout))

prediction = "isn't a dropout" if in_school['Target'] > dropout['Target'] else "is a dropout"
print("We predict this student is: " + prediction)

#display(school_interested)


In [ ]:
# Continue here

(p,r,f,s) = precision_recall_fscore_support(patientsy, y_pred, labels=[0,1])
print(f'precision={p}, recall={r}, f-score={f}, support={s}')